##LOAD HE DATASET

In [6]:
import pandas as  pd
data=pd.read_csv('/content/STEG_BILLING_HISTORY(1).csv')

#Import the neesary llibraries

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#EDA

In [17]:
data.types

AttributeError: 'DataFrame' object has no attribute 'types'

In [16]:
data.head()

,client_id,invoice_date,tarif_type,counter_number,counter_code,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type,counter_type_encoded
0,train_Client_0,2014-03-24,11,1335667.0,203,1.0,82.0,0.0,0.0,0.0,14302.0,14384.0,4.0,ELEC,0
1,train_Client_0,2013-03-29,11,1335667.0,203,1.0,1200.0,0.0,0.0,0.0,12294.0,13678.0,4.0,ELEC,0
2,train_Client_0,2015-03-23,11,1335667.0,203,1.0,123.0,0.0,0.0,0.0,14624.0,14747.0,4.0,ELEC,0
3,train_Client_0,2015-07-13,11,1335667.0,207,1.0,102.0,0.0,0.0,0.0,14747.0,14849.0,4.0,ELEC,0
4,train_Client_0,2016-11-17,11,1335667.0,207,1.0,572.0,0.0,0.0,0.0,15066.0,15638.0,4.0,ELEC,0


In [19]:
data.tail()

,client_id,invoice_date,tarif_type,counter_number,counter_code,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type,counter_type_encoded
12889,train_Client_100349,2012-05-21,11,339235.0,207,1.0,0.000000,0.0,0.0,0.0,18.000000,18.000000,4.0,ELEC,0
12890,train_Client_100349,2013-09-18,11,339235.0,207,1.0,0.000000,0.0,0.0,0.0,18.000000,18.000000,4.0,ELEC,0
12891,train_Client_100349,2015-01-26,11,339235.0,207,1.0,0.000000,0.0,0.0,0.0,18.000000,18.000000,4.0,ELEC,0
12892,train_Client_100349,2016-05-16,11,339235.0,207,1.0,0.000000,0.0,0.0,0.0,18.000000,18.000000,4.0,ELEC,0
12893,train_Client_100349,2018-01-22,11,339235.0,207,1.0,395.933685,0.0,0.0,0.0,15181.893741,15663.134026,4.0,ELEC,0


In [22]:
data.columns

Index(['client_id', 'invoice_date', 'tarif_type', 'counter_number',
       'counter_code', 'counter_coefficient', 'consommation_level_1',
       'consommation_level_2', 'consommation_level_3', 'consommation_level_4',
       'old_index', 'new_index', 'months_number', 'counter_type',
       'counter_type_encoded'],
      dtype='object')

#Checking the missing values

In [9]:
data.isnull().sum()

,0
client_id,0
invoice_date,0
tarif_type,0
counter_number,0
counter_statue,0
counter_code,0
reading_remarque,14
counter_coefficient,1
consommation_level_1,1
consommation_level_2,1


Handling the missing values

In [10]:
# Drop irrelevant or heavily missing columns if they exist
cols_to_drop = ['counter_statue', 'reading_remarque']
data.drop(columns=[col for col in cols_to_drop if col in data.columns], inplace=True)

# Fill categorical columns with mode
cat_cols = ['counter_code', 'counter_type', 'counter_number']
for col in cat_cols:
    if col in data.columns: # Check if column exists before filling
        data[col] = data[col].fillna(data[col].mode()[0])

# Fill numeric columns with mean
num_cols = ['counter_coefficient', 'consommation_level_1', 'consommation_level_2',
            'consommation_level_3', 'consommation_level_4', 'old_index',
            'new_index', 'months_number']
for col in num_cols:
    if col in data.columns: # Check if column exists before filling
        data[col] = data[col].fillna(data[col].mean())

#Confirming missig vakes

#Handling duplicates

In [11]:
data.duplicated().sum()

np.int64(0)

#Handling ouliers

In [20]:
# List of numeric columns
numeric_cols = [
    'counter_coefficient', 'consommation_level_1', 'consommation_level_2',
    'consommation_level_3', 'consommation_level_4',
    'old_index', 'new_index', 'months_number'
]

# Loop through each numeric column to handle outliers
for col in numeric_cols:
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Replace outliers with NaN
    data[col] = data[col].apply(lambda x: x if lower_bound <= x <= upper_bound else None)

    # Optionally fill NaN with median
    data[col] = data[col].fillna(data[col].median())

#Encoding

In [13]:
data.describe(include='object')

,client_id,invoice_date,counter_type
count,12894,12894,12894
unique,393,3059,2
top,train_Client_100345,2017-02-02,ELEC
freq,123,16,8979


#Encodng

In [14]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
data['counter_type_encoded'] = le.fit_transform(data['counter_type'])


In [21]:
data.drop(columns=['counter_statue'], inplace=True)


KeyError: "['counter_statue'] not found in axis"

# Convert 'invoice_date' to datetime objects

In [ ]:
data['invoice_date'] = pd.to_datetime(data['invoice_date'])
display(data['invoice_date'].head())

## Model (linear regression)

FEATURES AND TARGET

In [23]:
X = [
    'tarif_type',         # type of billing plan (encoded later)
    'counter_code',       # type of counter used
    'counter_coefficient',
    'consommation_level_1',
    'consommation_level_2',
    'consommation_level_3',
    'consommation_level_4',
    'old_index',          # previous meter reading
    'months_number',      # number of billing months
    'counter_type_encoded'  # encoded version of counter_type
]
Y ='new_index'


In [24]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
data['tarif_type'] = le.fit_transform(data['tarif_type'])
data['counter_code'] = le.fit_transform(data['counter_code'])


#sPLIT AND TRAIN

In [25]:
from sklearn.model_selection import train_test_split

X = data[[
    'tarif_type', 'counter_code', 'counter_coefficient',
    'consommation_level_1', 'consommation_level_2',
    'consommation_level_3', 'consommation_level_4',
    'old_index', 'months_number', 'counter_type_encoded'
]]

y = data['new_index']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


#Train random forest

In [27]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [28]:
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [30]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)

print("📊 Model Evaluation Metrics:")
print(f"MAE:  {mae:.2f}")
print(f"MSE:  {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²:   {r2:.2f}")

📊 Model Evaluation Metrics:
MAE:  295.07
MSE:  3889430.73
RMSE: 1972.16
R²:   0.95
